In [10]:
import itertools
import math

from vector_v5 import Vector
from vector2d_v3 import Vector2d

""" 기존 클래스에 함수를 포함하기 위한 데코레이터 """
def extend(klass):
    def decorator(func):
        #print(func.__name__, func, "was added to class", klass.__name__)
        setattr(klass, func.__name__, func)
        return func
    return decorator

# 연산자 오버로딩: 제대로 하기
이 장에서는 다음과 같은 내용을 설명한다. 
+ 파이선이 다른 자료형의 피연산자로 중위 연산자를 지원하는 방법
+ 다양한 자료형의 피연산자를 다루기 위한 덕 타이핑이나 명시적인 자료형 검사의 사용
+ 향상된 비교 연산자의 별난 행동
+ 피연산자를 처리할 수 없다고 ㅈ우위 연산자 메서드가 알려주는 방법
+ +=과 같은 계산 할당 연산자의 기본처리 방식 및 오버로딩 방법

10장의 Vector 클래스에 연산자 오버로딩을 적용하는 것으로 시작한다.

## 13.1 연산자 오버로딩 기본 지식

연산자 오버로딩은 남용되지 않으면 코드의 가독성이 향상되고 만족스러운 API를 구현할 수 있다. 파이선은 다음과 같은 제한을 두어 융통성, 사용성, 안정성을 적절히 유지한다.
+ 내장 자료형에 대한 연산자는 오버로딩할 수 없다.
+ 새로운 연산자를 생성할 수 없으며, 기존 연산자를 오버로딩만 할 수 있다.
+ is, and, or, not 연산자는 오버로딩할 수 없다(그러나 &, |, ~비트 연산자는 가능하다.)

## 13.2 단항 연산자

+ -(\_\_neg\_\_)    : 단항 산술 뺄셈
+ +(\_\_pos\_\_)    : 단항 산술 덧셈
+ ~(\_\_invert\_\_) : 정수형의 비트 반전 ~x는 -(x+1)로 정의된다.

\_\_invert\_\_( ) 메서드는 구현하지 않을 것이므로 Vector 객체에 ~v 연산을 실행하면 TypeError가 발생한다.

In [11]:
"""[예제 13-1] 예제 10-16에 단항 연산자 -와 + 추가 """ 

@extend(Vector)
def __abs__(self):
    return math.sqrt(sum(x * x for x in self))

@extend(Vector)
def __neg__(self):
    return Vector(-x for x in self)

@extend(Vector)
def __pos__(self):
    return Vector(self)

v1 = Vector([1, 2, 3, 4, 5])

print(+v1, -v1, abs(v1))
print(~v1) # __invert__( ) 구현이 필요 

(1.0, 2.0, 3.0, 4.0, 5.0) (-1.0, -2.0, -3.0, -4.0, -5.0) 7.416198487095663


TypeError: bad operand type for unary ~: 'Vector'

In [12]:
""" [예제 13-2] 이례적으로 x와 +x가 동일하지 않은 경우 """

# 1. decimal의 정밀도가 변하는 경우
import decimal
ctx = decimal.getcontext() # 현재 산술 콘텍스트 전역 설정에 대한 참조를 가져옴
ctx.prec = 40 # 정밀도 40
one_third = decimal.Decimal('1') / decimal.Decimal('3')
print(repr(one_third))
print(one_third == +one_third)

Decimal('0.3333333333333333333333333333333333333333')
True


In [13]:
ctx.prec = 39 # 정밀도 39
print(repr(one_third))
print(one_third == +one_third) # one_third는 40, +one_third는 39의 정밀도를 가진다

Decimal('0.3333333333333333333333333333333333333333')
False


In [14]:
# 2. Counter
from collections import Counter
ct = Counter('abracadabra')
print(repr(ct))
ct['r'] = -3
ct['d'] = 0
print(repr(ct))
print(repr(+ct)) # 음수나 0을 제거한다.

Counter({'a': 5, 'b': 2, 'r': 2, 'c': 1, 'd': 1})
Counter({'a': 5, 'b': 2, 'c': 1, 'd': 0, 'r': -3})
Counter({'a': 5, 'b': 2, 'c': 1})


## 13.3 벡터를 더하기 위해 + 오버로딩 하기
일반적으로 '+'은 시퀀즈 결합을, '*'는 시퀀스의 반복을 의미하나 여기서는 벡터의 수학 연산을 구현한다.

In [15]:
 """ [예제 13-4] Vector.add() # 1 """

@extend(Vector)
def __add__(self, other)    :
    """
    __add__() 메서드는 새로운 Vector 객체를 만들며, self나 other의 값을 변경하지 않음에 주의할 것
    """
    pairs = itertools.zip_longest(self, other, fillvalue=0.0)
    return Vector(a + b for a, b in pairs)

v1 = Vector([3, 4, 5, 6])
v2 = Vector([1, 2])

print(repr(v1 + v2))
print(v1 + v2 == Vector([3+1, 4+2, 5, 6]))
print(v1 + v2 == [3+1, 4+2, 5, 6])

Vector([4.0, 6.0, 5.0, 6.0])
True
True


예제 13-4에서 구현한 Vector 클래스를 사용하면 예제 13-5처럼 Vector 객체를 Vector2d 객체, 튜플, 그리고 숫자를 생성하는 어떠한 반복형에도 더할 수 있다. 그러나 13-6에서처럼 피연산자의 순서를 바꾸면 혼합된 덧셈 연산이 실패한다. 따라서 서로 다른 객체형에 대한 연산을 지원하기 위해 파이선은 중위 연산자의 특별 메서드에 아래 매커니즘으로 동작한다.

+ a에 __add__() 메서드가 정의되어 있으며 a.__add__(b)를 호출하고, 결과가 NotImplemented가 아닌면 반환한다. 
+ a에 __add__() 메서드가 정의되어 있지 않거나, 정의되어 있더라도 호출 후 NotImplemented가 반환되면, b에 __radd__() 메서드가 정의되어 있는지 확인해서 b.__add__(a)를 호출하고, 결과가 NotImplemented가 아닌면 반환한다.
+ b에 __radd__()가 정의되어 있지 않거나, 정의되어 있더라도 호출 후 NotImplemented가 반환되면, '지원하지 않은 피연자형'이라는 메시지와 함께 TypeError가 발생한다.

In [16]:
""" [에제 13-5] 비Vector 객체도 지원한다. """
v1 = Vector([3, 4, 5])
v1 + (10, 20, 30)
# 9장의 Vector2d와도 연산이 가능

Vector([13.0, 24.0, 35.0])

In [17]:
""" [에제 13-6] 왼쪽 피연산자가 비Vector 객체면 에러가 출력된다. """
v1 = Vector([3, 4, 5])
(10, 20, 30) + v1

TypeError: can only concatenate tuple (not "Vector") to tuple

In [18]:
""" [예제 13-7] __radd__()를 구현하면 다른 형의 피연산자가 오른쪽에 있는 연산 수행이 가능하다. """
@extend(Vector)
def __radd__(self, other):
    return self + other # 교환법칙이 성립하는 경우 이러한 방식으로 쉽게 구현이 가능하다.

(10, 20, 30) + v1

Vector([13.0, 24.0, 35.0])

추가적으로 비반복형 객체에 적용하면 __add__()는 예제 13-8과 같이 별로 도움되지 않는 메시지와 함께 에러가 발생한다. 더욱 심각한 문제는 연산자 특별 메서드가 자료형의 비호환성 문제 때문에 적절한 결과를 반환할 수 없을 때는 NotImplemeted 값을 반환해야지 TypeError 예외를 발생시키면 안된다. NotImplemeted를 반환함으로써 파이썬이 역순 메서드를 호출하려고 시도할 때 다른 피연산자 자료형의 구현자에 연산을 처리할 수 있는 기회를 줄 수 있기 때문이다. 따라서 벡터 덧셈을 수행하는 특별 메서드의 최종 버전은 예제 13-10과 같다.

In [19]:
""" [예제 13-10] + 연산자 메서드 최종 버전 """

@extend(Vector)
def __add__(self, other):
    try:
        pairs = itertools.zip_longest(self, other, fillvalue=0.0)
        return Vector(a+b for a, b in pairs)
    except TypeError:
        return NotImplemented
    
@extend(Vector)
def __radd__(self, other):
    return self + other # 교환법칙이 성립하는 경우 이러한 방식으로 쉽게 구현이 가능하다.   

In [20]:
v1 + 1

TypeError: unsupported operand type(s) for +: 'Vector' and 'int'

In [21]:
v1 + 'ABC'

TypeError: unsupported operand type(s) for +: 'Vector' and 'str'

## 13.4 벡터를 스칼라와 곱하기 위해 * 오버로딩하기
참고로 numpy 및 이와 유사한 라이브러리에서는 * 연산자를 사용하는 경우, 백터의 각 항목에 값을 곱하며, 벡터의 스칼라곱을 구하기 위해서는 numpy.dot( ) 함수를 사용한다. Vector에서도 '*' 연산자 오버로딩을 추가해 보자. 

[예제 13-10]에서 했던 것처럼 덕 타이핑 기법을 사용해서 \_\_mul\_\_() 안에서 TypeError를 잡을 수도 있지만, 여기서는 구스 타이핑 기법을 이용한 명시적인 방법을 사용하는 것이 타당하다. scalar의 자료형을 검사하기 위해 isinstance() 함수를 사용하지만 어떤 구체적인 자료형으로 하드코딩하는 대신 number.Real ABC로 검사한다. numbers.Real은 이 메서드에 필요한 자료형을 모두 포함할 뿐 아니라 numbers.Real ABC의 실제 서브클래스나 가상 서브클래스로 선언된 수치형 자료도 모두 포함한다.

In [22]:
""" [예제 13-11] * 연산자 메서드 추가 """
import numbers # 자료형을 검사하기 위해 numbers를 임포트한다.

@extend(Vector)
def __mul__(self, scalar):
    if isinstance(scalar, numbers.Real):        # scalar가 numbers.Real 서브클래스의 객체면 
        return Vector(n * scalar for n in self) # 항목 값들로 구성된 Vector 객체를 새로 생성한다.
    else:
        return NotImplemented

@extend(Vector)
def __rmul__(self, scalar):
    return self * scalar

v1 = Vector([1.0, 2.0, 3.0])
print(repr(14 * v1))
print(repr(14 * True))

from fractions import Fraction
print(repr(v1 * Fraction(1, 3)))

Vector([14.0, 28.0, 42.0])
14
Vector([0.3333333333333333, 0.6666666666666666, 1.0])


[표 13-1] 중위 연산자 메서드명. 인플레이스 연산자는 복합 할당에 사용된다. 

연산자|정방향|역순|인플레이스|설명
---|-----------|------------|------------|------------
 + | \_\_add\_\_( ) | \_\_radd\_\_( ) | \_\_iadd\_\_( ) | 덧셈이나 연결
 - | \_\_sub\_\_( ) | \_\_rsub\_\_( ) | \_\_isub\_\_( ) | 뺄셈 
 * | \_\_mul\_\_( ) | \_\_rmul\_\_( ) | \_\_imul\_\_( ) | 곱셈이나 반복
 / | \_\_truediv\_\_( ) | \_\_rtruediv\_\_( ) | \_\_itruediv\_\_( ) | 참 나눗셈
// | \_\_floordiv\_\_( ) | \_\_rfloordiv\_\_( ) | \_\_ifloordiv\_\_( ) | 플로어 나눗셈
 % | \_\_mod\_\_( ) | \_\_rmod\_\_( ) | \_\_imod\_\_( ) | 나머지 연산
 divmod( ) | \_\_divmod\_\_( ) | \_\_rdivmod\_\_( ) | \_\_idivmod\_\_( ) | 플로어 나눗셈의 몫과 나머지를 튜플로 반환
** | \_\_pow\_\_( ) | \_\_rpow\_\_( ) | \_\_ipow\_\_( ) | 누승
 @ | \_\_matmul\_\_( ) | \_\_rmatmul\_\_( ) | \_\_imatmul\_\_( ) | 행렬 곱셈(파이선 3.5에 추가됨)
 & | \_\_and\_\_( ) | \_\_rand\_\_( ) | \_\_iand\_\_( ) | 비트단위 곱(bitwise and)
\| | \_\_or\_\_( ) | \_\_ror\_\_( ) | \_\_ior\_\_( ) | 비트단위 합(bitwise or)
 ^ | \_\_xor\_\_( ) | \_\_rxor\_\_( ) | \_\_ixor\_\_( ) | 비트단위 배타합(bitwise xor)
<< | \_\_lshift\_\_( ) | \_\_rlshift\_\_( ) | \_\_ilshift\_\_( ) | 비트단위 왼쪽 시프트
>> | \_\_rshift\_\_( ) | \_\_rrshift\_\_( ) | \_\_irshift\_\_( ) | 비트단위 오른쪽 시프트
 

In [23]:
""" 파이선 3.5에성 추가된 @ 연산자 테스트 """
@extend(Vector)
def __matmul__(self, other):
    try:
        return sum(a * b for a, b in zip (self, other))
    except TypeError:
        return NotImplemented

@extend(Vector)    
def __rmatmul__(self, other):
    return self @ other

va = Vector([1, 2, 3])
vz = Vector([5, 6, 7])
print(va @ vz == 38.0)
print([10, 20, 30] @ vz)

True
380.0


## 13.5 향상된 비교 연산자
비교 연산자와 앞서 설명한 중위 연산자는 동작이 비슷하지만 아래와 같은 차이가 있다.
+ 표 13-2에 나열한 것처럼 정방향과 역순 연산자에 동일한 세트의 메서드가 사용된다. 예를 들어 == 연산자의 경우에는 정방향과 역순으로 실행하기 위해 인수만 바꿔서 동일한 \_\_eq\_\_( ) 메서드를 호출하지만, 정방향으로 \_\_gt\_\_( ) 메서드를 호출하는 경우, 역순으로는 인수를 바꿔서 \_\_lt\_\_( ) 메서드를 호출한다.
+ ==와 != 연산자의 경우 역순 메서드가 실패하면, 파이선은 TypeError를 발생시키는 대신 객체의 ID를 비교한다.

[표 13-2] 향상된 비교 연산자

종류|중위 연산자|정방향|역순|기본 처리
---|--------|----|---|--------
동치성| a==b | a.\_\_eq\_\_(b) | b.\_\_eq\_\_(a) | id(a)==id(b)를 반환한다.
    | a!=b | a.\_\_ne\_\_(b) | b.\_\_ne\_\_(a) | not(a==b)를 반환한다.
순서| a > b | a.\_\_gt\_\_(b) | b.\_\_lt\_\_(a) | TypeError를 발생시킨다.
순서| a < b | a.\_\_lt\_\_(b) | b.\_\_gt\_\_(a) | TypeError를 발생시킨다.
순서| a >= b | a.\_\_ge\_\_(b) | b.\_\_le\_\_(a) | TypeError를 발생시킨다.
순서| a >= b | a.\_\_le\_\_(b) | b.\_\_ge\_\_(a) | TypeError를 발생시킨다.

In [30]:
""" 기존의 \_\_eq\_\_( ) 함수의 동작을 개선해보자

def __eq__(self, other):
    return (len(self) == len(other) and
            all(a == b for a, b in zip(self, other)))
"""

va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
print(va == vb)

vc = Vector([1, 2]) 
v2d = Vector2d(1, 2)
print(vc == v2d) # 요소의 값이 같다면 Vector와 Vector2d도 동일하다고 판단한다.

t3 = (1, 2, 3)
print(va == t3) # 튜플이나 여타 반복형과도 동일하다고 판단한다.

True
True
False


피연산자를 평가할 때 지나친 자유분방함은 예기치 못한 결과를 낳을 수 있으며, 프로그래머는 예기치 못한 결과를 싫어한다. 파이선 자체를 보면 [1, 2] == (1, 2)는 거짓이다. 그러므로 보수적인 입장을 취하고 어느 정도 자료형을 검사하도록 변경해보자. 두 번째 피연산자가 Vector나 Vector 서브클래스의 객체인지 검사하고, 그렇다면 기존 __eq__() 메서드와 동일하게 처리한다. 그렇지 않은 경우에는 NotImplemented를 반환해서 파이선 인터프리터가 처리할 수 있게 해준다.

In [31]:
@extend(Vector)
def __eq__(self, other):
    if isinstance(other, Vector):
        return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))
    else:
        return NotImplemented
    
va = Vector([1.0, 2.0, 3.0])
vb = Vector(range(1, 4))
print(va == vb)

vc = Vector([1, 2]) 
v2d = Vector2d(1, 2)
print(vc == v2d) # 전과 동일한 결과가 나온다.

t3 = (1, 2, 3)
print(va == t3) # 원하는 대로 다른 결과가 나온다. 

True
True
False


Vector와 Vector2d 비교 결과에 대한 해석 
+ vc == v2d를 평가하기 위해 파이선은 Vector.\_\_eq\_\_(vc, v2d)를 호출한다.
+ Vector.\_\_eq\_\_(vc, v2d)는 v2d가 Vector 객체가 아님을 확인하고 NotImplemented를 반환한다.
+ NotImplemented가 반환되었으므로 파이선은 Vector2d.\_\_eq\_\_(v2d, vc)를 실행한다.
+ Vector2d.\_\_eq\_\_(v2d, vc)는 피연산자 두 개 모두를 튜플로 변환해서 비교한다. 따라서 결과가 True가 된다.

Vector와 tuple 비교 결과에 대한 해석 
+ va == t3를 평가하기 위해 파이선은 Vector.\_\_eq\_\_(va, t3)를 호출한다.
+ Vector.\_\_eq\_\_(va, t3)는 t3가 Vector 객체가 아님을 확인하고 NotImplemented를 반환한다.
+ NotImplemented가 반환되었으므로 파이선은 tuple.\_\_eq\_\_(t3, va)를 실행한다.
+ tuple.\_\_eq\_\_(t3, va)는 Vector형에 대해 알지 못하므로 NotImplemented를 반환한다.
+ == 연산자의 경우 특별히 역순 메서드가 NotImplemented를 반환하면, 파이선 인터프리터는 최후의 수단으로 두 객체의 ID를 비교한다.

!= 연산자는 object 클래스에서 상속한 \_\_ne\_\_( ) 메서드가 우리 목적에 맞게 처리해주므로, 우리가 직접 구현할 필요는 없다.

In [32]:
print(va != vb)
print(vc != v2d)
print(va != t3) # 원하는 대로 다른 결과가 나온다. 

False
False
True


## 13.6 복합 할당 연산자
우리가 구현한 Vector 클래스는 이미 +=과 *=복합 할당 연산자를 지원하고 있다. 인플레이스 연산자를 구현하지 않으면 복합 할당 연산자는 a+=b를 정확히 a=a+b와 동일하게 평가한다. 가변형의 경우 이렇게 동작하는 것이 타당하며, \_\_add\_\_() 메서드가 구현되어 있으면 아무런 코드를 추가하지 않고도 += 연산자가 동작한다. 

그러나 \_\_iadd\_\_() 등의 인플레이스 연산자 메서드를 정의한 경우에는 a+=b를 계산하기 위해 정의된 메서드가 호출된다. 이런 연산자는 새로운 객체를 생성하지 않고 왼쪽에서 나온 피연산자를 직접 변경한다.

In [33]:
""" [예제 13-18] BingoCage를 확장한 AddableBingoCage 클래스 """
import itertools

from tombola import Tombola
from bingo import BingoCage

class AddableBingoCage(BingoCage):
    def __add__(self, other):
        if isinstance(other, Tombola): # __add__() 메서드는 두 번째 피연산자가 Tombola 객체일 때만 작동한다.
            return AddableBingoCage(self.inspect() + other.inspect())
        else:
            return NotImplemented
    
    def __iadd__(self, other):
        if isinstance(other, Tombola):
            other_iterable = other.inspect()
        else:
            try:
                other_iterable = iter(other) # Tombola 객체가 아닐 때에는 other 반복자를 가져온다.
            except TypeError: # 실패하면 메시지와 함께 예외를 발생시킨다. 가능하면 에러 메시지에 사용자가 문제를 해결할 방법을 명학히 알려주는 것이 좋다.
                self_cls = type(self).__name__
                msg = "right operand in += must be {!r} or and iterable"
                raise TypeError(msg.format(self_cls))
        self.load(other_iterable)
        return self # 이 부분이 정말 중요하다. 할당 연산 특별 메서드는 반드시 self를 반환해야 한다. 

In [37]:
vowels = 'AEIOU'
globe = AddableBingoCage(vowels)
print(globe.inspect())
print(globe.pick() in vowels) # 항목을 하나 꺼내서 모음 문자인지 확인
print(len(globe.inspect()))

globe2 = AddableBingoCage('XYZ')
globe3 = globe + globe2
print(len(globe3.inspect()))

void = globe + [10, 20] # list를 더하려고 시도하면 TypeError가 발생하면서 실패한다. 이 에러 메시지는 __add__() 메서드가 
                        # NotImplemented를 반환한 후 파이썬 인터프리터가 생성한다.

('A', 'E', 'I', 'O', 'U')
True
4
7


TypeError: unsupported operand type(s) for +: 'AddableBingoCage' and 'list'

In [39]:
globe_orig = globe # globe이 복제가 일어나는지 확인하기 위해 현재 ID를 저장한다.
print(len(globe.inspect()))

globe += globe2
print(len(globe.inspect()))

globe += ['M', 'N'] # += 연산자의 오른쪽 피연산에는 어떠한 반복형이라도 올 수 있다.
print(len(globe.inspect()))

print(globe is globe_orig)

globe += 1 # 

9
12
14
True


TypeError: right operand in += must be 'AddableBingoCage' or and iterable

두 번째 피연산자의 측면에서 보면 += 연산자가 +연산자보다 자유롭다. + 연산자의 경우 서로 다른 자료형을 받으면 결과가 어떤 자료형이 되어야 하는지 혼란스러울 수 있으므로, 양쪽 피연산자가 동일한 자료형이기를 원한다. += 연산자의 경우에는 이러한 혼란이 없다. 왼쪽 객체의 내용이 갱신되므로, 연산 결과 자료형이 명확하다. (※ list의 경우도 + 연산은 list만 가능하지만 += 연산은 어떠한 반복형이라도 연결할 수 있다.)

\+ 연산은 추가로 \_\_radd\_\_( )가 필요없다. 동일한 자료형의 객체가 와야 작동하기 때문이다.